In [10]:
from web3 import Web3
import json
import numpy as np
from moralis import evm_api
import datetime
import pandas as pd
import plotly.express as px

In [11]:
decimals_USDC = 6
decimals_WETH = 18
tokenID = 29944

In [12]:
from constants import NFT_POSITIONMANAGER_CONTRACT_ADDRESS, NFT_POSITIONMANAGER_CONTRACT_ABI

NODE_PROVIDER='https://eth-mainnet.g.alchemy.com/v2/kqq6YvePSbVVpqEq_BeUTK9VOB8aO9e9'
web3_connection = Web3(Web3.HTTPProvider(NODE_PROVIDER))

NFT_POSITIONMANAGER_contract_abi = json.loads(NFT_POSITIONMANAGER_CONTRACT_ABI)
NFT_POSITIONMANAGER_contract = web3_connection.eth.contract(address=NFT_POSITIONMANAGER_CONTRACT_ADDRESS, abi=NFT_POSITIONMANAGER_contract_abi)

In [13]:
from constants import UNIV3_USDC_WETH_CONTRACT_ADDRESS, UNIV3_USDC_WETH_CONTRACT_ABI

UNIV3_USDC_WETH_contract_abi = json.loads(UNIV3_USDC_WETH_CONTRACT_ABI)
UNIV3_USDC_WETH_contract = web3_connection.eth.contract(address=UNIV3_USDC_WETH_CONTRACT_ADDRESS, abi=UNIV3_USDC_WETH_contract_abi)

We start by querying events so we query how much liquidity we add and at what block. Then we query when we decrease, how much liquidity we pull out.

In [14]:
latest = web3_connection.eth.get_block_number()
event_filter = NFT_POSITIONMANAGER_contract.events.IncreaseLiquidity.create_filter(fromBlock=0, toBlock=latest, argument_filters = {'tokenId': tokenID})
res = event_filter.get_all_entries()

In [15]:
event_filter = NFT_POSITIONMANAGER_contract.events.DecreaseLiquidity.create_filter(fromBlock=0, toBlock=latest, argument_filters = {'tokenId': tokenID})
res2 = event_filter.get_all_entries()

Call collect event. Collect event contains the funds we pulled out plus the interest. Some actions trigger events which are recorded on chain. Transactions on Eth scan.

In [16]:
if len(res2) == 0:
    endTime = datetime.datetime.now().replace(microsecond=0)
else:
    blockNumber_end = res2[-1]['blockNumber']
    liquidity_end = NFT_POSITIONMANAGER_contract.functions.positions(tokenID).call(block_identifier=blockNumber_end)[7]
    if liquidity_end == 0:
        endTime = datetime.datetime.fromtimestamp(web3_connection.eth.get_block(blockNumber_end).timestamp)
        event_filter = NFT_POSITIONMANAGER_contract.events.Collect.create_filter(fromBlock=0, toBlock=latest, argument_filters = {'tokenId': tokenID})
        res3 = event_filter.get_all_entries()
        amount0_burned = res2[-1]['args']['amount0']
        amount1_burned = res2[-1]['args']['amount1']
        amount0_collected = res3[-1]['args']['amount0']
        amount1_collected = res3[-1]['args']['amount1']
    else:
        endTime = datetime.datetime.now().replace(microsecond=0)

In [17]:
res

[AttributeDict({'args': AttributeDict({'tokenId': 29944,
   'liquidity': 642845769648974884,
   'amount0': 3824486509842431902924,
   'amount1': 7645663666271}),
  'event': 'IncreaseLiquidity',
  'logIndex': 350,
  'transactionIndex': 155,
  'transactionHash': HexBytes('0x85511f0cb38d1808dfdd3cdd5e5c3c713aa436dc98967ce54ad9c9ef1686e44f'),
  'address': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
  'blockHash': HexBytes('0x654ee650697ab1d23650f368e29a2fcbe93dcfbdf79fbc704400aea55aac9011'),
  'blockNumber': 12540024}),
 AttributeDict({'args': AttributeDict({'tokenId': 29944,
   'liquidity': 1609629190976361,
   'amount0': 7999999999998472686,
   'amount1': 22952036523}),
  'event': 'IncreaseLiquidity',
  'logIndex': 221,
  'transactionIndex': 174,
  'transactionHash': HexBytes('0x9f272b54ce441da52382716f33c1c304e20478c11e108ebaed662c663246e9a4'),
  'address': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
  'blockHash': HexBytes('0x280d6ad75c13da66ec9f208c5e9dc84c8de14a7ac9ccdf642d3ee6a4

In [18]:
res2

[]

Query transaction hash (view on etherscan) - view the events.


In [19]:
dates = [endTime]
blockNumber_start = res[0]['blockNumber']
startTime = datetime.datetime.fromtimestamp(web3_connection.eth.get_block(blockNumber_start).timestamp)
startDate = (startTime+datetime.timedelta(1)).replace(minute=0, hour=0, second=0, microsecond=0)
dates = list(pd.date_range(startDate, endTime).to_pydatetime()) + [endTime]
for result in res:
    mint_date = datetime.datetime.fromtimestamp(web3_connection.eth.get_block(result['blockNumber']).timestamp)
    dates.append(mint_date)
dates = np.sort(list(set(dates)))

In [20]:
fees_USDC = []
fees_WETH = []
liquidity_USDC = []
liquidity_WETH = []
sqrtPriceX96_array = []

for date in dates:

    #Moralis: Get blocknumber from date
    #https://docs.moralis.io/web3-data-api/reference/get-date-to-block

    api_key = "zWUKbN72Ae3VcnbnUzXUg0wfIHUglQ5pGpGElmOgM2eMypBmaRB0DWwr2tDhk9HI"
    params = {
        "date": date,
        "chain": "eth",
    }

    result = evm_api.block.get_date_to_block(
        api_key=api_key,
        params=params,
    )

    position = NFT_POSITIONMANAGER_contract.functions.positions(tokenID).call(block_identifier=result['block'])
    lowerTick = position[5]
    upperTick = position[6]
    liquidity = position[7]
    feeGrowthInside0LastX128 = position[8]
    feeGrowthInside1LastX128 = position[9]

    feeGrowthGlobal0X128 = UNIV3_USDC_WETH_contract.functions.feeGrowthGlobal0X128().call(block_identifier=result['block'])
    feeGrowthGlobal1X128 = UNIV3_USDC_WETH_contract.functions.feeGrowthGlobal1X128().call(block_identifier=result['block'])
    UNIV3_USDC_WETH_contract_lowerTick = UNIV3_USDC_WETH_contract.functions.ticks(lowerTick).call(block_identifier=result['block'])
    feeGrowthOutside0X128_LowerTick = UNIV3_USDC_WETH_contract_lowerTick[2]
    feeGrowthOutside1X128_LowerTick = UNIV3_USDC_WETH_contract_lowerTick[3]
    UNIV3_USDC_WETH_contract_upperTick = UNIV3_USDC_WETH_contract.functions.ticks(upperTick).call(block_identifier=result['block'])
    feeGrowthOutside0X128_UpperTick = UNIV3_USDC_WETH_contract_upperTick[2]
    feeGrowthOutside1X128_UpperTick = UNIV3_USDC_WETH_contract_upperTick[3]
    slot0 = UNIV3_USDC_WETH_contract.functions.slot0().call(block_identifier=result['block'])
    sqrtPriceX96 = slot0[0]
    currentTick = slot0[1]

    if currentTick >= lowerTick:
        feeGrowthBelow0X128 = feeGrowthOutside0X128_LowerTick
        feeGrowthBelow1X128 = feeGrowthOutside1X128_LowerTick
    else:
        feeGrowthBelow0X128 = feeGrowthGlobal0X128 - feeGrowthOutside0X128_LowerTick
        feeGrowthBelow1X128 = feeGrowthGlobal1X128 - feeGrowthOutside1X128_LowerTick

    if currentTick < upperTick:
        feeGrowthAbove0X128 = feeGrowthOutside0X128_UpperTick
        feeGrowthAbove1X128 = feeGrowthOutside1X128_UpperTick
    else:
        feeGrowthAbove0X128 = feeGrowthGlobal0X128 - feeGrowthOutside0X128_UpperTick
        feeGrowthAbove1X128 = feeGrowthGlobal1X128 - feeGrowthOutside1X128_UpperTick

    fees0X128 = (feeGrowthGlobal0X128 - feeGrowthBelow0X128 - feeGrowthAbove0X128 - feeGrowthInside0LastX128) % 2**256
    fees1X128 = (feeGrowthGlobal1X128 - feeGrowthBelow1X128 - feeGrowthAbove1X128 - feeGrowthInside1LastX128) % 2**256

    tokensOwed0 = position[10]
    tokensOwed1 = position[11]

    fees_USDC.append(fees0X128/2**128*liquidity/10**decimals_USDC + tokensOwed0/10**decimals_USDC)
    fees_WETH.append(fees1X128/2**128*liquidity/10**decimals_WETH + tokensOwed1/10**decimals_WETH)

    if currentTick < lowerTick:
        liquidity_token0 = liquidity*(1/np.sqrt(1.0001**lowerTick) - 1/np.sqrt(1.0001**upperTick))
        liquidity_token1 = 0
    elif currentTick >= upperTick:
        liquidity_token0 = 0
        liquidity_token1 = liquidity*(np.sqrt(1.0001**upperTick) - np.sqrt(1.0001**lowerTick))
    else:
        liquidity_token0 = liquidity*(1/(sqrtPriceX96/2**96) - 1/np.sqrt(1.0001**upperTick))
        liquidity_token1 = liquidity*(sqrtPriceX96/2**96 - np.sqrt(1.0001**lowerTick))

    liquidity_USDC.append(liquidity_token0/10**decimals_USDC)
    liquidity_WETH.append(liquidity_token1/10**decimals_WETH)

    sqrtPriceX96_array.append(sqrtPriceX96)

In [21]:
fees_daily = pd.DataFrame()
fees_daily["Date"] = dates
fees_daily["Fees USDC"] = fees_USDC
fees_daily["Fees WETH"] = fees_WETH
fees_daily["Liquidity USDC added"] = 0
fees_daily["Liquidity WETH added"] = 0
fees_daily["Liquidity USDC"] = liquidity_USDC
fees_daily["Liquidity WETH"] = liquidity_WETH



for result in res:
    mint_date = datetime.datetime.fromtimestamp(web3_connection.eth.get_block(result['blockNumber']).timestamp)
    fees_daily.loc[fees_daily["Date"] == mint_date, "Liquidity USDC added"] = result['args']['amount0']/10**decimals_USDC
    fees_daily.loc[fees_daily["Date"] == mint_date, "Liquidity WETH added"] = result['args']['amount1']/10**decimals_WETH



if len(res2) > 0:
    if liquidity_end == 0:
        #update values for the block when LP is removed
        fees_daily.loc[fees_daily["Date"] == endTime, "Fees USDC"] = (amount0_collected - amount0_burned)/10**decimals_USDC
        fees_daily.loc[fees_daily["Date"] == endTime, "Fees WETH"] = (amount1_collected - amount1_burned)/10**decimals_WETH
        fees_daily.loc[fees_daily["Date"] == endTime, "Liquidity USDC"] = amount0_burned/10**decimals_USDC
        fees_daily.loc[fees_daily["Date"] == endTime, "Liquidity WETH"] = amount1_burned/10**decimals_WETH

fees_daily["Price WETH/USDC"] = 1/((np.array(sqrtPriceX96_array)/2**96)**2 * 10**(decimals_USDC-decimals_WETH))
fees_daily["Liquidity added USDC converted"] = fees_daily["Liquidity USDC added"] + fees_daily["Liquidity WETH added"] * fees_daily["Price WETH/USDC"]
fees_daily["Portfolio USDC converted"] = fees_daily["Fees USDC"] + fees_daily["Liquidity USDC"] + (fees_daily["Fees WETH"] + fees_daily["Liquidity WETH"])  * fees_daily["Price WETH/USDC"]
fees_daily["Fees USDC converted"] = fees_daily["Fees USDC"] + fees_daily["Fees WETH"] * fees_daily["Price WETH/USDC"]

In [22]:
display(fees_daily)

,Date,Fees USDC,Fees WETH,Liquidity USDC added,Liquidity WETH added,Liquidity USDC,Liquidity WETH,Price WETH/USDC,Liquidity added USDC converted,Portfolio USDC converted,Fees USDC converted
0,2021-05-31 04:42:49,2.187491e+50,2.187491e+38,3.824487e+15,7.645664e-06,0.0,0.000020,2296.253502,3824486509842432.0,2187490805635688860667901626474600349488242701...,2187490805635688860667901626474600349488242701...
1,2021-06-01 00:00:00,2.187491e+50,2.187491e+38,0.000000e+00,0.000000e+00,0.0,0.000020,2701.014068,0.0,2187490806521098844021448524643503858213051338...,2187490806521098844021448524643503858213051338...
2,2021-06-01 17:29:52,2.192968e+50,2.192968e+38,8.000000e+12,2.295204e-08,0.0,0.000020,2532.747089,7999999999998.472656,2192968090476593644342733809085677933103514365...,2192968090476593644342733809085677933103514365...
3,2021-06-02 00:00:00,2.192968e+50,2.192968e+38,0.000000e+00,0.000000e+00,0.0,0.000020,2631.472703,0.0,2192968090693095807993688809746988332443738864...,2192968090693095807993688809746988332443738864...
4,2021-06-03 00:00:00,2.192968e+50,2.192968e+38,0.000000e+00,0.000000e+00,0.0,0.000020,2707.78983,0.0,2192968090860456828024224153373255267205105361...,2192968090860456828024224153373255267205105361...
...,...,...,...,...,...,...,...,...,...,...,...
798,2023-07-03 00:00:00,2.602518e+50,2.602518e+38,0.000000e+00,0.000000e+00,0.0,0.000024,1937.157934,0.0,2602518230788659067033447156036224300642568671...,2602518230788659067033447156036224300642568671...
799,2023-07-04 00:00:00,2.602518e+50,2.602518e+38,0.000000e+00,0.000000e+00,0.0,0.000024,1954.625283,0.0,2602518230834118249105542596103324219664450117...,2602518230834118249105542596103324219664450117...
800,2023-07-05 00:00:00,2.602518e+50,2.602518e+38,0.000000e+00,0.000000e+00,0.0,0.000024,1936.373349,0.0,2602518230786617040524922579026475827046212991...,2602518230786617040524922579026475827046212991...
801,2023-07-06 00:00:00,2.602518e+50,2.602518e+38,0.000000e+00,0.000000e+00,0.0,0.000024,1911.517794,0.0,2602518230721930160110049648071612056457670022...,2602518230721930160110049648071612056457670022...


In [23]:
fees_daily[['Liquidity USDC added']].sum()

Liquidity USDC added    4.542208e+15
dtype: float64

In [24]:
fees_daily[['Liquidity WETH added']].sum()

Liquidity WETH added    0.000009
dtype: float64

In [25]:
fees_daily.to_csv(f'{tokenID}.csv')